In [10]:
import requests
import re
from bs4 import BeautifulSoup
import bs4
import string
import re
from pprint import pprint
import csv
import time
import pandas as pd
from functools import partial
import math
from time import sleep
import random
import os

In [1]:
def crawl_homophone():
    begin = "https://www.homophone.com/search?page=" 
    end = "&q&type=begin"
    lines = []

    for i in range(1,87):

        # create url & obrain request
        url = begin + str(i) + end
        print(url)
        result = requests.get(url)
        beauty_result = BeautifulSoup(result.text)

        # parse request
        list_of_cards = beauty_result.find_all('div', attrs={'class':'card'}) # ok safe now

        for card in list_of_cards:
            line = []
            words = card.find_all('a', attrs={'class':'btn word-btn'})
            for word in words:
                line.append(word.getText())
            if len(line) >=2:
                lines.append(line)
    return lines

In [2]:
def write_to_file(filename, data):
    with open(filename, "w") as file:
        for line in data: # for each set of homonyms
            to_write = ""
            # check that there is at least 2 homonyms...
            if len(line) < 2:
                continue
            # create one line    
            for i, word in enumerate(line): 
                to_write += word
                if i+1 < len(line):
                    to_write += "/"
            to_write += '\n'
            # save line to file
            file.write(to_write)

In [3]:
import re

# remove phonetic translation ....
def strip_between_brackets(line):
    r = line.strip()
    regex = re.compile(".(?![^\(]*\))") # skip between  ()  ... but not working as expected ... ')' remains
    result = re.findall(regex, r)
    r = "".join(result)
    r= r.replace(")", "")  # and is solved here
    
    rm1 = '\['
    rm2 = '\]'
    regex = re.compile(".(?![^" + rm1 + "]*" + rm2 + ")")  # are [_doublet_] 
    result = re.findall(regex, r)
    r = "".join(result)
    r= r.replace("]", "")  # and is solved here
    
    r = r.strip()
    r = r.strip(".")
    r = r.strip()
    return r 

In [4]:
# parse all files ... one by one...
path = "scraped-homonyms/8-oronyms.txt"

w_lines = []

tmp = []

with open(path, 'r') as file:
    for line in file:
        
        line = line.strip() #strip_between_brackets(line)
        
        if "Stressed vowel" in line:
            continue
        if "Lexical set" in line:
            continue
        if "Sound:" in line:
            continue
        if "See also" in line:
            continue   
        if "Nonte:" in line:
            continue 
        if " split" in line:
            continue
            
#         line = line.strip()
        if line.startswith("http:"):
            continue
        if line.startswith("https:"):
            continue                
        else: # line == 1
            tmp.append(line)
            
        words = line.split("\t")
        w_line = []
        for word in words:
            word = word.strip()
            if len(word) > 0:
                w_line.append(word)
        if len(w_line) > 1:    
            w_lines.append(w_line) 
            
            
# and write it
write_to_file("scraped-homonyms/8.txt",w_lines)

#  Merge all files to one

In [76]:
# create list of sets
list_of_sets = []

# read file by file
for i in range(1,9): # files 1..8
    fname = "scraped-homonyms/"+str(i)+".txt"
    print(fname)
    with open(fname, "r") as file: # for each line
        for line in file:
            line = line.strip()
            line = line.split('/')
            spliteed_set = set(line) # read homonym group
            
            added = False
            for ss in list_of_sets:
                if ss.intersection(spliteed_set):
                    added = True
                    ss = ss.union(spliteed_set)
            if not added:
                list_of_sets.append(spliteed_set)

# writing part
write_to_file("homonyms-data/scraped-merged.txt", list_of_sets)

scraped-homonyms/1.txt
scraped-homonyms/2.txt
scraped-homonyms/3.txt
scraped-homonyms/4.txt
scraped-homonyms/5.txt
scraped-homonyms/6.txt
scraped-homonyms/7.txt
scraped-homonyms/8.txt
